In [ ]:
import numpy as np
import pathlib
import pydicom
import os
from pydicom.pixel_data_handlers.util import apply_modality_lut
from scipy import ndimage
from skimage import morphology
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow import keras
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras import layers

In [ ]:
IMG_SIZE = (224, 224)
NUM_EPOCHS = 2
SPLIT = 0.15
DATASET_PATH = r'C:\Users\adais\Documents\CovidX CT\divided data'
#TRAIN_PATH = r'D:\Python\Covid NN\ct-scan-transfer-learning\DIVIDED DATASET\train'
#VAL_PATH = r'D:\Python\Covid NN\ct-scan-transfer-learning\DIVIDED DATASET\val'

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs["val_accuracy"]
        train_acc = logs["accuracy"]
        if val_acc >= self.threshold and train_acc >= self.threshold:
            self.model.stop_training = True

acc_callback = MyThresholdCallback(0.99)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(directory=DATASET_PATH,
                                                                    shuffle=True,
                                                                    validation_split=SPLIT,
                                                                    image_size=IMG_SIZE,
                                                                    color_mode="rgb", label_mode="categorical",
                                                                    labels="inferred", seed=15,
                                                                    subset="training", batch_size=32)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(directory=DATASET_PATH,
                                                                    shuffle=True,
                                                                    validation_split=SPLIT,
                                                                    image_size=IMG_SIZE,
                                                                    color_mode="rgb", label_mode="categorical",
                                                                    labels="inferred", seed=15,
                                                                    subset="validation", batch_size=32)
base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

outputs = base_model.output
outputs= layers.AveragePooling2D((2,2),strides=(2,2))(outputs)
outputs = layers.Flatten(name="flatten")(outputs)
outputs = layers.Dense(1024, activation="relu")(outputs)
outputs = layers.Dropout(0.2)(outputs)
outputs = layers.Dense(512, activation="relu")(outputs)
outputs = layers.Dropout(0.2)(outputs)
outputs = layers.Dense(64, activation="relu")(outputs)
outputs = layers.Dense(2, activation="softmax")(outputs)
final_model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

final_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
final_model.summary()
history = final_model.fit(train_dataset.take(50),
                           validation_data=val_dataset.take(50),
                           epochs=NUM_EPOCHS, callbacks=[acc_callback], batch_size=32)

fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x in range(NUM_EPOCHS)],
                         y=history.history['accuracy'],
                         mode='lines',
                         name='Training',
                         ))
fig.add_trace(go.Scatter(x=[x for x in range(NUM_EPOCHS)],
                         y=history.history['val_accuracy'],
                         mode='lines',
                         name='Validation'))

fig.update_layout(title='VGG19', xaxis_title='Epoch', yaxis_title='Accuracy')
#fig.write_html("history_plot_VGG19_final_Sao Paulo.html")
history_dict = history.history
#json.dump(history_dict, open('VGG19_training_log_Sao Paulo', 'w'))
fig.show()

In [ ]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

def VizGradCAM(model, image, interpolant=0.5, plot_results=True):
    assert (
        interpolant > 0 and interpolant < 1
    ), "Heatmap Interpolation Must Be Between 0 - 1"

    last_conv_layer = next(
        x for x in model.layers[::-1] if isinstance(x, K.layers.Conv2D)
    )
    target_layer = model.get_layer(last_conv_layer.name)
    original_img = image
    img = np.expand_dims(original_img, axis=0)
    prediction = model.predict(img)

    # Obtain Prediction Index
    prediction_idx = np.argmax(prediction)

    # Compute Gradient of Top Predicted Class
    with tf.GradientTape() as tape:
        gradient_model = Model([model.inputs], [target_layer.output, model.output])
        conv2d_out, prediction = gradient_model(img)
        # Obtain the Prediction Loss
        loss = prediction[:, prediction_idx]

    gradients = tape.gradient(loss, conv2d_out)
    output = conv2d_out[0]
    weights = tf.reduce_mean(gradients[0], axis=(0, 1))
    activation_map = np.zeros(output.shape[0:2], dtype=np.float32)

    for idx, weight in enumerate(weights):
        activation_map += weight * output[:, :, idx]

    # Resize
    activation_map = cv2.resize(
        activation_map.numpy(), (original_img.shape[1], original_img.shape[0])
    )
    activation_map = np.maximum(activation_map, 0)

    # Convert to 0/255
    activation_map = (activation_map - activation_map.min()) / (
        activation_map.max() - activation_map.min()
    )
    activation_map = np.uint8(255 * activation_map)

    # Convert to Heatmap
    heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)

    # Superimpose Heatmap on Image Data
    original_img = np.uint8(
        (original_img - original_img.min())
        / (original_img.max() - original_img.min())
        * 255
    )
    cvt_heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    plt.rcParams["figure.dpi"] = 100
    return cvt_heatmap

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(np_image):
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   #np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [ ]:
DATASET_ABSOLUTE_PATH = r"C:\Users\adais\Documents\COVID DICOM\manifest-1608266677008\MIDRC-RICORD-1A\MIDRC-RICORD-1A-419639-000421\12-05-2006-NA-CT CHEST WITHOUT CONTRAST-50477"

# Will load images from directories, convert them to pixel arrays with HU values
# and return array of arrays (distinct images)

heat_maps = []

def load_dicom_dataset(directory):
    dataset_to_return = []
    dataset_path = pathlib.Path(directory)
    for data_sample in dataset_path.iterdir():
        single_sample_list = []
        if os.path.isfile(pathlib.Path(data_sample)):
            continue
        img_index = 0
        for image in data_sample.iterdir():
            source_image_file = pydicom.dcmread(pathlib.Path(image), force=True)
            image_array = source_image_file.pixel_array      
            #image_array = image_array[::2, ::2]
            # PREDICTION
            if 110 > img_index > 100:
                heat_maps.append(VizGradCAM(final_model, load(image_array)))
            
            hu_image_array = apply_modality_lut(image_array, source_image_file)
            masked_image = remove_data_noise(hu_image_array)
            
            single_sample_list.append(list(masked_image))
            img_index += 1
        dataset_to_return.append(single_sample_list)
    return dataset_to_return

In [ ]:
def remove_data_noise(hu_image): # Masking the image
    hu_image = crop_image(hu_image, 350, 340)

    segmentation = morphology.dilation(hu_image, np.ones((1, 1)))
    labels, label_nb = ndimage.label(segmentation)
    label_count = np.bincount(labels.ravel().astype(np.int))
    label_count[0] = 0

    mask = labels == label_count.argmax()

    mask = morphology.dilation(mask, np.ones((1, 1)))
    mask = ndimage.morphology.binary_fill_holes(mask)
    mask = morphology.dilation(mask, np.ones((3, 3)))

    masked_image = mask * hu_image
    return masked_image

def crop_image(image, cropx, cropy): # Cropping image equally from both sides
    y,x = image.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return image[starty:starty + cropy, startx:startx + cropx]

def plot_3d_image(heat_map_array): # #3D plotting
#     z, x, y = data_array.nonzero()
#     marker_data = go.Scatter3d(
#         x=x, y=y, z=z,
#         marker=go.scatter3d.Marker(size=1),
#         opacity=0.15,
#         mode='markers', marker_color='rgba(39, 39, 245, 0.8)'
#     )
    z, x, y = heat_map_array.nonzero()
    heat_marker_data = go.Scatter3d(
        x=x, y=y, z=z,
        marker=go.scatter3d.Marker(size=1),
        opacity=0.15,
        mode='markers', marker_color='rgba(245, 39, 56, 0.8)'
    )
    fig = go.Figure(data=heat_marker_data)
    fig.update_layout(width=700, margin=dict(r=10, l=10, b=10, t=10))
    fig.update_layout(scene_aspectmode='cube')
    fig.show()


In [ ]:
heat_maps = []

dataset = load_dicom_dataset(DATASET_ABSOLUTE_PATH)

average_shape_array = np.mean(np.array([patient for patient in dataset]), axis=0)
upper_hu_bound = average_shape_array < -620
lower_hu_bound = average_shape_array > -740
average_shape_array = np.where((lower_hu_bound == upper_hu_bound), upper_hu_bound, 0)


heat_maps_gray = [np.mean(x, axis=2) for x in heat_maps]
heat_maps_gray = [cv2.resize(x, dsize=(340, 350), interpolation=cv2.INTER_CUBIC) for x in heat_maps_gray]
heat_maps_gray = np.array(heat_maps_gray)
print(average_shape_array)

#plot_3d_image(heat_maps_gray)